In [49]:
import pandas

df_train = pandas.read_csv('./50_dataset.csv', quotechar="\'")
df_test = pandas.read_csv('./test.csv', quotechar="\'")

In [50]:
import re
import string
import demoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer

demoji.download_codes()
def clean_tweets(data):
    new_data = data.copy()

    username_hash = r'[#@]\w+'
    punctuation = '[%s]+' % re.escape(string.punctuation)
    special_char = r'[^0-9a-zA-Z\s]+'
    number = r'[0-9]+'
    space = r'\s{2,}'
    space_begin_end = r'^\s+|\s+$'
    url = r'(https?|www):\/{1,}\w+\W+\w+\/{1,}\w+'
    char_ref = r'&\w+;'

    for i in range(len(new_data)):
        new_data[i] = re.sub(char_ref, ' ', new_data[i])
        new_data[i] = re.sub(username_hash, '', new_data[i])
        new_data[i] = re.sub(url, '', new_data[i])
        new_data[i] = re.sub(punctuation, '', new_data[i])
        new_data[i] = re.sub(number, '', new_data[i])
        new_data[i] = re.sub(space_begin_end, '', new_data[i])
        new_data[i] = re.sub(space, '', new_data[i])
        new_data[i] = demoji.replace(new_data[i], '')
        new_data[i] = re.sub(special_char, '', new_data[i])

    return new_data

def case_fold(data):
    new_data = data.copy()
    return list(map(lambda s: s.lower(), new_data))

def tokenize(data):
    new_data = data.copy()
    return list(map(lambda s: s.split(' '), new_data))

def stem(data):
    new_data = data.copy()
    stemmer = StemmerFactory().create_stemmer()

    return list(map(lambda s: stemmer.stem(s), new_data))

def get_term(data):
    new_data = data.copy()
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(new_data)

    return count_vectorizer.get_feature_names()

... OK (Got response in 0.47 seconds)
Writing emoji data to /Users/bangkodir/.demoji/codes.json ...
... OK


In [51]:
tweet_train = stem(case_fold(clean_tweets(df_train['tweet'])))

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(tweet_train, df_train['category'])

vect = TfidfVectorizer()
vect.fit(tweet_train)

X_train_tfidf = vect.transform(X_train)
X_test_tfidf = vect.transform(X_test)

In [53]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svm_c = SVC(kernel='linear')
svm_c.fit(X_train_tfidf, y_train)
pred = svm_c.predict(X_test_tfidf)

print(accuracy_score(pred, y_test))

report = classification_report(y_test, pred, output_dict=True)
print('positive: ', report['positif'])
print('negative: ', report['negatif'])

0.8461538461538461
positive:  {'precision': 1.0, 'recall': 0.3333333333333333, 'f1-score': 0.5, 'support': 3}
negative:  {'precision': 0.8333333333333334, 'recall': 1.0, 'f1-score': 0.9090909090909091, 'support': 10}


In [54]:
# TODO : test our model here